# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key



# Google developer API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df=pd.read_csv("cities_info.csv")
cities_df


,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,rikitea,-23.1203,-134.9692,75.128,58,80,6.85,PF,1621111783
1,omboue,-1.5746,9.2618,78.350,89,75,2.76,GA,1621111968
2,punta arenas,-53.1500,-70.9167,46.670,93,90,5.14,CL,1621111779
3,voznesenye,61.0106,35.4781,49.424,99,100,1.34,RU,1621111968
4,port alfred,-33.5906,26.8910,64.220,83,89,2.82,ZA,1621111772
...,...,...,...,...,...,...,...,...,...
561,gornyye klyuchi,45.2500,133.5000,50.234,91,12,2.30,RU,1621112287
562,chumikan,54.7183,135.3150,33.422,99,100,4.92,RU,1621112287
563,panambi,-28.2925,-53.5017,75.470,50,40,2.06,BR,1621112288
564,ishinomaki,38.4167,141.3000,62.870,82,75,5.66,JP,1621112288


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations=cities_df[["Lat", "Lng"]]
locations
Humidity=cities_df["Humidity"]    
len(locations)

566

In [4]:
Humidity
max_humidity=cities_df["Humidity"].max()
max_humidity

100

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:

  # A max temperature lower than 80 degrees but higher than 70.

  # Wind speed less than 10 mph.

  # Zero cloudiness.

  # Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.


hotel_df=cities_df.loc[(cities_df["Max_Temp"]>70)&(cities_df["Max_Temp"]<80)&(cities_df["Windspeed"]<5)&(cities_df["Cloudiness"]==0)]
len(hotel_df)                       
hotel_df                   

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Windspeed,Country,Date
6,cabra,37.4725,-4.4421,75.470,35,0,2.06,ES,1621112096
71,tiznit,29.5833,-9.5000,76.172,34,0,1.40,MA,1621112117
126,arraial do cabo,-22.9661,-42.0278,73.670,68,0,2.06,BR,1621111788
138,concordia,-31.3930,-58.0209,70.070,56,0,3.09,AR,1621112138
139,sao joao da barra,-21.6403,-41.0511,77.270,61,0,4.12,BR,1621111908
152,maxixe,-23.8597,35.3472,70.304,87,0,4.06,MZ,1621112140
212,adeje,28.1227,-16.7260,71.870,64,0,4.63,ES,1621112162
227,abha,18.2164,42.5053,70.070,56,0,3.60,SA,1621112025
240,imbituba,-28.2400,-48.6703,71.870,64,0,2.06,BR,1621111886
299,belmonte,-15.8631,-38.8828,75.470,69,0,2.06,BR,1621112195


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df["Hotel_Name"]=""
hotel_df.head()
    

C:\Users\Saumy\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel_Name
6,cabra,37.4725,-4.4421,75.470,35,0,2.06,ES,1621112096,
71,tiznit,29.5833,-9.5000,76.172,34,0,1.40,MA,1621112117,
126,arraial do cabo,-22.9661,-42.0278,73.670,68,0,2.06,BR,1621111788,
138,concordia,-31.3930,-58.0209,70.070,56,0,3.09,AR,1621112138,
139,sao joao da barra,-21.6403,-41.0511,77.270,61,0,4.12,BR,1621111908,


In [8]:
# set up a parameters dictionary
parameters = {
    "type": "lodging",
    "radius": 5000,
    "key": g_key
}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Use the lat/lng we recovered to identify cities
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    parameters["location"] = f"{lat},{lng}"


    try:
    # make request and print url
        response = requests.get(base_url, params=parameters)
    
    # convert to json
        data = response.json()
        #print(data)
 

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    
        hotel_df.loc[index, "Hotel_Name"] = data["results"][0]["name"]
        print(data["results"][0]["name"])
    except (KeyError, IndexError):
 
       print("Missing field/result... skipping.")
hotel_df

C:\Users\Saumy\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


MS Fuente Las Piedras
Missing field/result... skipping.
Pousada Porto Praia
Palmar Hotel Casino
Pousada Porto De Canoas
Oceano
Gran Tacande Wellness & Relax
Abha Hotel
Praia Hotel Imbituba
Pousada Monte Carmelo O Pão
مقبرة ال مجمل
Quality Suites Vila Velha
Hotel Morotin
Seven Corner View Homestay
B &R Rest House
Green House
龙海钻石大酒店
Railway motel
Kuba Lodge
Piyaporn Hill Paradise Hotel
Oceano


,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel_Name
6,cabra,37.4725,-4.4421,75.470,35,0,2.06,ES,1621112096,MS Fuente Las Piedras
71,tiznit,29.5833,-9.5000,76.172,34,0,1.40,MA,1621112117,
126,arraial do cabo,-22.9661,-42.0278,73.670,68,0,2.06,BR,1621111788,Pousada Porto Praia
138,concordia,-31.3930,-58.0209,70.070,56,0,3.09,AR,1621112138,Palmar Hotel Casino
139,sao joao da barra,-21.6403,-41.0511,77.270,61,0,4.12,BR,1621111908,Pousada Porto De Canoas
152,maxixe,-23.8597,35.3472,70.304,87,0,4.06,MZ,1621112140,Oceano
212,adeje,28.1227,-16.7260,71.870,64,0,4.63,ES,1621112162,Gran Tacande Wellness & Relax
227,abha,18.2164,42.5053,70.070,56,0,3.60,SA,1621112025,Abha Hotel
240,imbituba,-28.2400,-48.6703,71.870,64,0,2.06,BR,1621111886,Praia Hotel Imbituba
299,belmonte,-15.8631,-38.8828,75.470,69,0,2.06,BR,1621112195,Pousada Monte Carmelo O Pão


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:

# Display figure
# Create bank symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Add marker layer ontop of heat map



marker_locations = locations

markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))